In [1]:
-- Gloria Budiman
-- EE569 HW4
-- 6528-1836-50
-- 11/25/2016

require 'nn'
require 'optim'

torch.manualSeed(0)
torch.setnumthreads(4)

In [2]:
function setup(threshold) 
    classes = { 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 }
    geometry = { 32, 32 }

    net = nn.Sequential()

    net:add(nn.SpatialConvolution(1, 6, 5, 5))
    
    net:add(nn.AddConstant(threshold))
    net:add(nn.ReLU())
    net:add(nn.AddConstant(-threshold))
    
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.SpatialConvolution(6, 16, 5, 5))
    
    net:add(nn.AddConstant(threshold))
    net:add(nn.ReLU())
    net:add(nn.AddConstant(-threshold))
    
    net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

    net:add(nn.View(16*5*5))
    net:add(nn.Linear(16*5*5, 120))
    
    net:add(nn.AddConstant(threshold))
    net:add(nn.ReLU())
    net:add(nn.AddConstant(-threshold))
    
    net:add(nn.Linear(120, 84))
    
    net:add(nn.AddConstant(threshold))
    net:add(nn.ReLU())
    net:add(nn.AddConstant(-threshold))
    
    net:add(nn.Linear(84, #classes))
    net:add(nn.LogSoftMax())
    
    parameters, gradParameters = net:getParameters()
    criterion = nn.ClassNLLCriterion()
    confusion = optim.ConfusionMatrix(classes)
end

In [3]:
function normalize(input, n_channels)
    local mean = {}
    local stdev = {}
    
    for channel = 1, n_channels do
        mean[channel] = input.data[{ {}, {channel}, {}, {} }]:mean()
        stdev[channel] = input.data[{ {}, {channel}, {}, {} }]:std()
        
        print('Channel ' .. channel .. ' mean: ' .. mean[channel] .. ' stdev: ' .. stdev[channel])
        
        input.data[{ {}, {channel}, {}, {} }]:add(-mean[channel])
        input.data[{ {}, {channel}, {}, {} }]:div(stdev[channel])
    end
end

In [4]:
function load_data()    
    train = torch.load('mnist-p1b-train.t7')
    test = torch.load('mnist-p1b-test.t7')
    
    n_train = train.data:size()[1]
    n_test = test.data:size()[1]
    
    train.data = train.data:double()
    test.data = test.data:double()
end

In [5]:
function exec_training(obj, n_channels, current_epoch) 
    confusion:zero()
    
    for t = 1, n_train, batch_size do
        local limit = math.min(t + batch_size - 1, n_train)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local feval = function(x)
            collectgarbage()
            
            if x ~= parameters then
                parameters:copy(x)
            end
            gradParameters:zero()
            
            local outputs = net:forward(inputs)
            local f = criterion:forward(outputs, targets)
            local df_do = criterion:backward(outputs, targets)
            net:backward(inputs, df_do)
            
            for i = 1, limited_batch_size do
                confusion:add(outputs[i], targets[i])
            end
            
            return f, gradParameters
        end
        
        sgd_state = sgd_state or {
            -- default parameter
        }
        optim.sgd(feval, parameters, sgd_state)
    end
    
    confusion:updateValids()
    return confusion.averageValid
end

In [6]:
function exec_testing(obj, n_channels, current_epoch)
    confusion:zero()
    
    for t = 1, n_test, batch_size do
        local limit = math.min(t + batch_size - 1, n_test)
        
        local limited_batch_size = limit - t + 1
        local inputs = torch.Tensor(limited_batch_size, n_channels, geometry[1], geometry[2])
        local targets = torch.Tensor(limited_batch_size)
        local k = 1
        
        for i = t, limit do
            local input = obj.data[i]:clone()
            local target = obj.label[i]
            
            inputs[k] = input
            targets[k] = target
            k = k + 1
        end
        
        local preds = net:forward(inputs)
        
        for i = 1, limited_batch_size do
            confusion:add(preds[i], targets[i])
        end
    end 
    
    confusion:updateValids()
    return confusion.averageValid
end

In [7]:
function cycle(threshold)
    io.write(string.format('Cycling with threshold %.1f', threshold))
    setup(threshold)
    load_data()
    normalize(train, 1)
    normalize(test, 1)

    n_epoch = 32
    batch_size = 256
    local last_train_conf
    local last_test_conf
    for epoch = 1, n_epoch do
        local acc_train = exec_training(train, 1, epoch)

        if (epoch == n_epoch) then
            last_train_conf = confusion:__tostring__()
        end

        local acc_test = exec_testing(test, 1, epoch)

        if (epoch == n_epoch) then
            last_test_conf = confusion:__tostring__()
        end

        io.write(string.format('Epoch %3d: %.4f%% | %.4f%%\n', epoch, acc_train * 100, acc_test * 100))
    end

    print(last_train_conf)
    print(last_test_conf)
end

In [8]:
for _threshold = -1, 1, 0.1 do
    cycle(_threshold)
end

Cycling with threshold -1.0

Channel 1 mean: 25.509416422526 stdev: 70.180423838273	


Channel 1 mean: 25.87140625 stdev: 70.739298703524	


Epoch   1: 10.0626% | 10.0000%


Epoch   2: 10.0295% | 10.0000%


...

Epoch  28: 92.6080% | 93.3937%


Epoch  29: 92.7675% | 93.5589%


Epoch  30: 92.9395% | 93.6719%


Epoch  31: 93.0954% | 93.8348%


Epoch  32: 93.2432% | 93.9328%
ConfusionMatrix:
[[    5743       2      17      16       6      32      39       9      43      16]   96.961% 	[class: 1]
 [       0    6559      44      19       7      22       6      15      60      10]   97.286% 	[class: 2]
 [      37      45    5502      67      81      11      53      89      63      10]   92.346% 	[class: 3]
 [      32      38      95    5574       1     149      21      63     103      55]   90.915% 	[class: 4]
 [       4      21      44       3    5429       2      64      10      22     243]   92.931% 	[class: 5]
 [      56      38      16     106      34    4952      98      14      56      51]   91.348% 	[class: 6]
 [      45      24      29       3      42      86    5670       0      18       1]   95.809% 	[class: 7]
 [      16      42      75      12      43      12       0    5887       8     170]   93.966% 	[class: 8]
 [      23      79      39     148      23      91      40      12    5307      89]   90.702% 	[class: 9